In [15]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [16]:
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

@njit
def rolling_mean_14(x):
    return rolling_mean(x, window_size=14)
@njit
def rolling_mean_30(x):
    return rolling_mean(x, window_size=30)

In [17]:
def format_df_to_mlforecast(df, date_col, target_col, unique_id='mean'):
    df_ = df.rename({
        date_col: "ds",
        # target_col: 'y',
    }, axis=1)

    df_['ds'] = pd.to_datetime(df_['ds'])

    df_['y'] = df_[target_col].copy()
    # df_.drop(columns=target_col)

    df_['unique_id'] = unique_id
    return df_

In [18]:
selected_sensors_df = pd.read_csv("../data/selected_sensors2_cleaned.csv", index_col=0)

In [19]:
TEST_START_DATE = "2019-04-02"
scenarios_sensors = {
    # 0: 1, 4372603
    # "0_12M_train_7M_test": {"train_start": "2017-03-25", "train_end": "2018-03-25", "test_start": "2018-03-26", "test_end": "2018-10-10"},
    '2': {
        # "18M_train":  {"train_start": "2017-04-01", "train_end": "2018-10-01"},
        # "12M_train":  {"train_start": "2017-04-01", "train_end": "2018-04-01", "val_start": "2017-04-01", "val_end": "2018-04-01"},
        # "12M_train_3M_val":  {"train_start": "2017-04-01", "train_end": "2018-04-01", "val_start": "2018-04-01", "val_end": "2018-07-01"},
        "12M_train_6M_val":  {"train_start": "2017-04-01", "train_end": "2018-04-01", "val_start": "2018-04-01", "val_end": "2018-10-01"},
        "12M_train_9M_val":  {"train_start": "2017-04-01", "train_end": "2018-04-01", "val_start": "2018-04-01", "val_end": "2019-01-01"},
        "12M_train_12M_val":  {"train_start": "2017-04-01", "train_end": "2018-04-01", "val_start": "2018-04-01", "val_end": "2019-04-01"},
        },
}
scenarios_sensors['5'] = scenarios_sensors['2'].copy()
scenarios_sensors['6'] = scenarios_sensors['2'].copy()

In [20]:
from MLForecastPipeline import *

In [21]:
def full_split_data(df, scenario, test_start_date=TEST_START_DATE, date_col="ds"):
    train_data = df[df[date_col] <= scenario['train_end']]
    val_data = df[(df[date_col] > scenario['val_start']) & (df[date_col] <= scenario['val_end'])]
    test_data = df[df[date_col] >= test_start_date]
    return train_data, val_data, test_data

def split_data(df, scenario, date_col="ds"):
    """Extracts train and test data based on train end date."""
    train_data = df[df[date_col] <= scenario['train_end']]
    test_start = pd.to_datetime(scenario['train_end']) + pd.Timedelta(days=1)
    test_data = df[df[date_col] >= test_start]
    return train_data, test_data

models = {
    "SGD_Optuna": SGDRegressor( penalty='elasticnet', l1_ratio=0.5, alpha=0.001, random_state=42 ),
    # "SGDRegressor": SGDRegressor(random_state=42),
    # "SGD_ElasticNet": SGDRegressor( penalty='elasticnet', l1_ratio=0.5, alpha=0.001, random_state=42 ),
}

# Define lag transformations
from mlforecast.lag_transforms import *
lag_transforms_options = [
    # {},
    {1: [rolling_mean_14], 7: [rolling_mean_30], 30: [expanding_mean]},
    {1: [expanding_mean], 7: [rolling_mean_14], 30: [expanding_mean]},
    # {7: [RollingMean(window_size=7)], 30: [RollingMean(window_size=30)], 60: [RollingMean(window_size=60)], },
    {7: [RollingMean(7), RollingStd(7)], 30: [RollingMean(30)], 60: [ExpandingMean()], 14: [ExponentiallyWeightedMean(alpha=0.3)],},
    {7: [RollingMean(7), RollingStd(7), ExpandingStd()], 14: [RollingMean(14), ExpandingStd(), ExponentiallyWeightedMean(alpha=0.3)], 30: [RollingMean(30)], 60: [ExpandingMean()],},
]

In [22]:
# Reshaping to MLForecast format
def format_multi_df_to_mlforecast(df):
    df_melted = df.melt(id_vars=['full_date'], var_name='unique_id', value_name='y')
    return df_melted.rename(columns={'full_date': 'ds'})

In [23]:
def optuna_objective(trial, train_df, test_df, transforms, lags, lag_transforms):
    alpha = trial.suggest_float('alpha', 1e-6, 1, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)
    max_iter = trial.suggest_int('max_iter', 300, 1000, step=100)  # Optimizing max_iter (number of iterations)
    eta0 = trial.suggest_float('eta0', 1e-6, 1, log=True)
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)

    model = SGDRegressor(alpha=alpha, l1_ratio=l1_ratio, max_iter=max_iter, eta0=eta0, tol=tol, penalty='elasticnet', random_state=42)

    try:
        fcst = MLForecast(
            models=[model],
            freq='D',
            lags=lags,
            target_transforms=transforms,
            lag_transforms=lag_transforms,
            num_threads=1,
        )
        fcst.fit(train_df)
        predictions = fcst.predict(h=len(test_df))
        mape = mape_met(test_df['y'].values, predictions['SGDRegressor'].values)
        return mape
    except Exception as e:
        print(e)
        return float('inf')
    
import optuna

def run_optuna_search(train_df, test_df, transforms, lags, lag_transforms, n_trials=30):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: optuna_objective(trial, train_df, test_df, transforms, lags, lag_transforms), n_trials=n_trials)
    return study.best_params


In [24]:
from joblib import Parallel, delayed
import time

def process_scenario(sensor_name, scenario_name, scenario, selected_sensors_df, models, lag_transforms_options, ratios=[0.33, 0.66, 1]):
    """ Process each scenario independently and save results. """
    print(f'{sensor_name}_{scenario_name}')
    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]
    
    _, val_test_df = split_data(formatted_df, scenario) # everything after train
    train_df, val_df, test_df = full_split_data(formatted_df, scenario) # here different validation set but same test 
    optimal_lags_list = get_optimal_lags(train_df, 'y', ratios=ratios, low_resources=True)
    target_transforms = get_dynamic_transforms(train_df, remove_boxcox=True)

    print(len(target_transforms))
    print(len(optimal_lags_list))
    print(len(models))
    print(len(lag_transforms_options))

    results = evaluate_models_sgd_tune(train_df, val_df, val_test_df, models, target_transforms, lag_transforms_options, optimal_lags_list, n_jobs=-1, n_trials=30)

    # Save results
    save_results(results, f"results/run_18/{sensor_name}_{scenario_name}.csv")

    return results

def run_all_scenarios_parallel(scenarios_sensors, selected_sensors_df, models, lag_transforms_options, ratios=[0.33, 0.66, 1]):
    # don't use all cpus (instead all but one)
    results = Parallel(n_jobs=15)( 
        delayed(process_scenario)(sensor_name, scenario_name, scenario, selected_sensors_df, models, lag_transforms_options, ratios=ratios)
        for sensor_name, scenarios in scenarios_sensors.items()
        for scenario_name, scenario in scenarios.items()
    )

    return results

In [25]:
run_all_scenarios_parallel(scenarios_sensors, selected_sensors_df, models, lag_transforms_options)

KeyboardInterrupt: 

In [26]:
for sensor_name, scenarios in scenarios_sensors.items():
    for scenario_name, scenario in scenarios.items():
        process_scenario(sensor_name, scenario_name, scenario, selected_sensors_df, models, lag_transforms_options)

2_12M_train_6M_val


[I 2025-04-09 08:59:35,810] A new study created in memory with name: no-name-ed21a705-6f20-4a91-9ce9-87dd5dd59dfc


5
3
1
4
Total model fits to run: 4320
0/4320 Training SGD_Optuna with transforms (), lags [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 18, 20, 22, 23, 24, 25, 29, 33, 41, 44, 57, 67, 69, 79, 80, 82, 83, 94, 105, 107, 116, 127, 129, 132, 137, 141, 148, 153, 154, 156, 159, 160, 162, 163, 167, 169, 170, 171, 172, 173, 175, 177, 178, 180, 183, 185, 187, 188], and lag_transforms {1: [<function rolling_mean_14 at 0x000001FD2B418E00>], 7: [<function rolling_mean_30 at 0x000001FD2B44A2A0>], 30: [<function expanding_mean at 0x000001FD24420720>]}...


c:\Users\PC314\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
[I 2025-04-09 08:59:37,882] Trial 12 finished with value: inf and parameters: {'alpha': 0.0007315965577640477, 'l1_ratio': 0.587193593523315, 'max_iter': 500, 'eta0': 0.018293531866598486, 'tol': 5.173663090714001e-06}. Best is trial 12 with value: inf.
c:\Users\PC314\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
c:\Users\PC314\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
c:\Users\PC314\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
c:\Users\PC314\anaconda3\Lib\site-packages\mlforecast\core.py:625: UserWarning: Found null values in lag1, rolling_mean_14_lag1.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
[I 20

Input X contains infinity or a value too large for dtype('float64').
Input X contains NaN.
SGDRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
Input X contains infinity or a value too large for dtype('float64').


...

Input X contains infinity or a value too large for dtype('float64').
Input X contains infinity or a value too large for dtype('float64').


c:\Users\PC314\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
[I 2025-04-09 08:59:54,963] Trial 25 finished with value: inf and parameters: {'alpha': 2.8076525280989703e-06, 'l1_ratio': 0.006301223376877246, 'max_iter': 700, 'eta0': 2.846187512602953e-05, 'tol': 5.220398504011821e-05}. Best is trial 0 with value: inf.


Input X contains infinity or a value too large for dtype('float64').


[I 2025-04-09 08:59:56,394] Trial 3 finished with value: 906.5438091881477 and parameters: {'alpha': 0.0007665875136460211, 'l1_ratio': 0.298088906058554, 'max_iter': 300, 'eta0': 2.8483582134075847e-06, 'tol': 0.00047735338094288967}. Best is trial 3 with value: 906.5438091881477.
[I 2025-04-09 08:59:56,580] Trial 4 finished with value: 1165.6515220209108 and parameters: {'alpha': 5.941145027993189e-05, 'l1_ratio': 0.533145097608076, 'max_iter': 400, 'eta0': 1.0953066748900995e-06, 'tol': 1.2844817827753353e-05}. Best is trial 3 with value: 906.5438091881477.
[I 2025-04-09 08:59:56,697] Trial 22 finished with value: 861.0545277761358 and parameters: {'alpha': 4.143779642168948e-06, 'l1_ratio': 0.6585917012615998, 'max_iter': 1000, 'eta0': 1.6046116066323753e-06, 'tol': 3.053507216546032e-06}. Best is trial 22 with value: 861.0545277761358.
[I 2025-04-09 08:59:56,795] Trial 28 finished with value: 884.3018207464805 and parameters: {'alpha': 0.01313731156098636, 'l1_ratio': 0.0525728037

KeyboardInterrupt: 